<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 521 Files and Databases - Fall 2016</h1>
<h1 style="font-size: 250%;">Problem Set #3</h1>
<h1 style="font-size: 200%;">Version 1.0</h1>
<h1>Issued Monday 10/03/2016; Due Monday 10/17/2016</h1>
</div>

**Homework Information:** The answer cells in this notebook contain metadata that enable it to be automatically parsed. Thus, you must enter your answers *only* into the cells provided. Do not delete any answer cell and/or add a new one to replace it. If you do so by accident, download a new copy of the problem set's notebook and transfer your answers into it. You can introduce new 'coding' cells in an effort to test your approaches to problems, however, these introduced cells will not be considered when grading.

Finally, some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of 70.71% for each class meeting after their due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Fill in the fields as described
class Student:
    first = "Cunyuan"
    last = "Huang"
    onyen = "hcuny"
    pid = "730041361"

In this problem set we will revisit the *Monkepo* data discussed in Lecture 3. You may want to revist that lecture to reaquaint yourself with the data set. Your first task will be to construct and populate a database with the following schema.

 <table style="border:none;"><tbody>
        <tr style="border:none;">
            <td style="padding:0px 20px;border:none;" colspan="2">
            <table><tbody>
            <tr><th colspan="6" style="text-align:center;">Monkepo</th></tr>
            <tr><td><u>mid</u></td>
                <td>name</td>
            </tr>
            </tbody></table>
            </td>
            <td style="padding:20px 20px;border:none;">
            <table><tbody>
            <tr><th colspan="4" style="text-align:center;">Appearance</th></tr>
            <tr><td><u>mid</u></td>
                <td><u>latitude</u></td>
                <td><u>longitude</u></td>
                <td><u>datetime</u></td>
            </tr>
            </tbody></table>
            </td>
            <td style="padding:20px 20px;border:none;">
            <table><tbody>
            <tr><th colspan="4" style="text-align:center;">Class</th></tr>
            <tr><td><u>mid</u></td>
                <td><u>major (1/0)</u></td>
                <td>type</td>
            </tr>
            </tbody></table>
            </td>
        </tr>
        </tbody></table>

The *Monkepo* relation should hold just the *unique* name of a *monkepo* species and an autoincremented primary key, *mid*. When a *Monkepo* appears its location and appearance time are stored as a tuple in the *Appearance* relation. Finally, the *Class* relation is used to hold information about the major and minor class of the monkepo species. The major class is indicated by a value of *1* in the major field, and the minor class by a *0*. There should be no class entry for a *None* minor type.  

Use this verison of the <a href="http://csbio.unc.edu/mcmillan/Media/monkepo.csv" download>data file</a> to fill your database (it is different than the one used in lecture 3).

---
**Part 1:** In the cell provided below write a Python fragment to create the specified database, and name it *"monkepo.db"* (Hint: you should investigate Python's *DateTime* module to find methods for combining the date and time values from the data file into a single attribute). You code should commit and close your database.

In [1]:
import sqlite3
import pandas as pd

monkfile=pd.read_csv('Y://Data//DB//monkepo.csv')

lst_name=monkfile['name'].tolist()
lst_majorclass=monkfile['majorclass'].tolist()
lst_minorclass=monkfile['minorclass'].tolist()
lst_latitude=monkfile['latitude'].tolist()
lst_longitude=monkfile['longitude'].tolist()
#lst_datetime=monkfile['date'].str.cat(monkfile['time'],sep=' ').tolist()
lst_datetime=(monkfile['date']+' '+monkfile['time']).tolist()

conn=sqlite3.connect('Y://Data//DB//monkepo.db')

cur = conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS Monkepo;
DROP TABLE IF EXISTS Appearance;
DROP TABLE IF EXISTS Class;

CREATE TABLE Monkepo (mid INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT UNIQUE);

CREATE TABLE Appearance (mid INTEGER , latitude REAL, longitude INTEGER, datetime TEXT, 
                         FOREIGN KEY(mid) REFERENCES Monkepo(mid), PRIMARY KEY (mid, latitude, longitude, datetime));
                         
CREATE TABLE Class (mid INTEGER, major INTEGER, type TEXT, 
                    FOREIGN KEY(mid) REFERENCES Monkepo(mid), PRIMARY KEY (mid, major)); 
                      ''')


# Monkepo Relation
unique_name=[]
for i in range(len(lst_name)):
    if lst_name[i] in unique_name:
        continue
    else:
        unique_name.append(lst_name[i])
        cur.execute('INSERT INTO Monkepo (name) VALUES (?)',(lst_name[i],))

#Appearance Relation    
mid_app=[]    
for i in range(len(lst_name)):
    cur.execute('SELECT mid FROM Monkepo WHERE name = ? ', (lst_name[i], ))
    mid_app.append(cur.fetchone()[0])
    
lst_tuple=[(mid_app[i],lst_latitude[i],lst_longitude[i],lst_datetime[i]) for i in range(len(lst_name))]

lst_tuple_unique=list(set(lst_tuple))

cur.executemany('''INSERT INTO Appearance
        (mid , latitude , longitude, datetime) 
        VALUES ( ?, ?, ?, ? )''',  (lst_tuple_unique) )


#CLass Relation

unique_name1=[]
for i in range(len(lst_name)):
    if lst_name[i] in unique_name1:
        continue
    else:
        unique_name1.append(lst_name[i])
        cur.execute('SELECT mid FROM Monkepo WHERE name = ? ', (lst_name[i], ))
        mid_cur=cur.fetchone()[0]        
        cur.execute('INSERT INTO Class(mid, major, type) VALUES (?,?,?)',(mid_cur,1, lst_majorclass[i]))
        if  lst_minorclass[i]=='None':
            continue
        else:
            cur.execute('INSERT INTO Class(mid, major, type) VALUES (?,?,?)',(mid_cur,0, lst_minorclass[i]))
        


conn.commit()
cur.close()

---
**Part 2:** Using the database that you created in part 1, write Python function to count the frequency of appearances of each Monkepo species. Your function should take the name of your database as an argument and return a sorted list of tuples (Monkepo.name, #appearances) with the most frequent first. Implement this function in using a dictionary like we did in Lecture 3.

In the cell below, enter the definition of your function, followed by a call to it, where the result is displayed via a Pandas data frame as shown in the code fragment supplied.

In [2]:
import sqlite3
import pandas

def freq_count1(db_name):
    conn=sqlite3.connect(db_name)
    cur = conn.cursor()    
    species_count={}
    for row in cur.execute('''SELECT m.name from Appearance a
                           inner join Monkepo m
                           on a.mid=m.mid 
                           '''):
        if row[0] not in species_count:
            species_count[row[0]] = 1
        else:
            species_count[row[0]] += 1
    
    
    data=list(species_count.items())
    data.sort(key=lambda tup: tup[1], reverse=True) #学习一个
    df=pandas.DataFrame(data, columns=["Name", "Rating"])
    
    return df
    

freq_count1('Y://Data//DB//monkepo.db')

,Name,Rating
0,PigDye,24445
1,AttaRat,18661
2,RopeSaw,10397
3,DewEel,8717
4,Veeee,7356
5,NovaNet,4766
6,TarPiece,4036
7,TubaZ,3639
8,ARasp,2593
9,AnIronAd-,2497


---
**Part 3:** Implement a second version of your function to count the frequency of appearances of each Monkepo species, but this time use SQL's "GROUP BY" directive. The input and output of this version should be the same as the one you implemented in Part 2. Display your answer using a Pandas data frame as well.

In [3]:
import sqlite3
import pandas

def freq_count2(db_name):
    conn=sqlite3.connect(db_name)
    cur = conn.cursor()
    listOfTuples=[]
    for row in cur.execute('''SELECT name, count(*) from Appearance a
                           inner join Monkepo m
                           on a.mid=m.mid 
                           group by m.mid
                           order by count(*) DESC'''):
        listOfTuples.append(row)
    
    df=pandas.DataFrame(listOfTuples, columns=["Name", "Rating"])
    
    return df

freq_count2('Y://Data//DB//monkepo.db')

,Name,Rating
0,PigDye,24445
1,AttaRat,18661
2,RopeSaw,10397
3,DewEel,8717
4,Veeee,7356
5,NovaNet,4766
6,TarPiece,4036
7,TubaZ,3639
8,ARasp,2593
9,AnIronAd-,2497


---
**Part 4:** Use Jupyter's [*%timeit*](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) function to compare the performance of the two functions that you wrote in parts 2 and 3.

In [4]:
%timeit freq_count1('Y://Data//DB//monkepo.db')
%timeit freq_count2('Y://Data//DB//monkepo.db')

# conclusion: the first function takes more time per loop, the second function takes less time 
#each loop, thus SQL language is faster here.

10 loops, best of 3: 138 ms per loop
10 loops, best of 3: 37.3 ms per loop


---
**Part 5:** In the following cell write a Python function that finds both the rarest and most  common monkepo species for each class. Your function should take your database name as an input and it it should return a list with with a tuple for each distinct class type and columns with the most and least common Monkepo species names.

In [5]:
import sqlite3
import pandas

def class_species(db_name):
    conn=sqlite3.connect(db_name)
    cur = conn.cursor()   
    listOfTuples=[]
    
    for row in cur.execute('''select max_table.type, max_species, min_species from
                             (select type, name max_species from
                             (select c.type, a.mid, m.name, count(*) species_num from 
                             Appearance a inner join Class c 
                             on a.mid=c.mid
                             inner join Monkepo m
                             on m.mid=c.mid
                             group by c.type, c.mid) p
                             group by type
                             having species_num=max(species_num))  max_table
                             inner join
                             (select type, name min_species from
                             (select c.type, a.mid, m.name, count(*) species_num from 
                             Appearance a inner join Class c 
                             on a.mid=c.mid
                             inner join Monkepo m
                             on m.mid=c.mid
                             group by c.type, c.mid) p
                             group by type
                             having species_num=min(species_num))  min_table
                             on max_table.type=min_table.type'''):
        listOfTuples.append(row)
    df=pandas.DataFrame(listOfTuples, columns=["Type", "most_common","least_common"])
    return df
        
        

class_species('Y://Data//DB//monkepo.db')

,Type,most_common,least_common
0,Aggressive,YankMe,IonHelment
1,Beastly,TrainId,RaidOrgan
2,Boring,PigDye,Asniper
3,Dirty,HerdSwans,XiNo
4,Elfish,ClayFire,WiffyGlutG
5,Freezing,Xjyn,Xjyn
6,Haunting,Stylag,Nagger
7,Hot,TigerHowl,AlienSent
8,Insect,DewEel,FerretTube
9,Metallic,GimmeATen,GimmeATen


---
**Part 6:** One of the reasons that Monkepo appear is that they depend on supply stations in order to survive in Earth's harsh environment. The same smartphone app that Millennials use for detecting Mokepos also detects the locations of these supply stations. A list of detected stations is available for <a href="http://csbio.unc.edu/mcmillan/Media/stations.csv" download>download</a> here.

In the next cell you should add a new table to your database called *SupplyStations*. Make sure that you commit and close the modified database.

Then write a code fragment that finds, for each supply station how many Monkepo appearances were closer to it than to any other station. Your code should produce a list of tuples (stationID, latitude, longitude, #ofNearbyMonkepo) sorted from most popular to least popular.

***This problem is challenging, allow for enough time to do it!*** You can approximate the distance between a Monkepo appearance and a supply station by $\sqrt{(latitude_{Monkepo} - latitude_{Station})^2 + (longitude_{Monkepo} - longitude_{Station})^2}$. This is not techically correct, but it is close at the scales and latitudes we are considering.

In [37]:
#Enter your Python 2 code here
import sqlite3
import pandas as pd

#-------------------------------------------------------------------------------------------------------------
#PART1 : create new table SupplyStations
#-------------------------------------------------------------------------------------------------------------
stations=pd.read_csv('Y://Data//DB//stations.csv')
Id_lst=stations['stationID'].tolist()
lst_Id=[int(t) for t in Id_lst]
lat_lst=stations['latitude'].tolist()
lst_lat=[float(t) for t in lat_lst]
long_lst=stations['longitude'].tolist()
lst_long=[float(t) for t in long_lst]

conn=sqlite3.connect('Y://Data//DB//monkepo.db')
cur = conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS SupplyStations;

CREATE TABLE SupplyStations (stationID INTEGER PRIMARY KEY, latitude REAL, longitude REAL);
                  ''')

for i in range(len(Id_lst)):
    cur.execute('''
                INSERT INTO SupplyStations
                (stationID, latitude , longitude) 
                VALUES (?,?,?)
                ''', (lst_Id[i],lst_lat[i],lst_long[i]))
    
conn.commit()
cur.close()

#-----------------------------------------------------------------------------------------------------------
# PART2: get list of tuples (stationID, latitude, longitude, #ofNearbyMonkepo) 
#-----------------------------------------------------------------------------------------------------------
conn=sqlite3.connect('Y://Data//DB//monkepo.db')
cur = conn.cursor()

lst_app=[]      #list of tuples from Appearance table

for row in cur.execute("""select mid, latitude, longitude
                          from Appearance
                       """):
    lst_app.append(row)
conn.commit()

lst_station=[] #list of tuples from SupplyStations table
for row in cur.execute("""select stationID, latitude, longitude
                          from SupplyStations
                       """):
    lst_station.append(row)    
conn.commit()    

lst_mid_sid=[] #create a new list of tuples (mid, stationID, latitude, longitude) for each appearance with the nearest station
for ele in lst_app:
    dist=[]
    dist=[((ele[1]-sta[1])**2+(ele[2]-sta[2])**2)**0.5 for sta in lst_station]
    fmin=min(dist)
    p=dist.index(fmin)
    lst_mid_sid.append((ele[0],lst_station[p][0],lst_station[p][1],lst_station[p][2]))    


conn.commit()
cur.close()

db = sqlite3.connect(':memory:') #create a temporary database and temporary table Mid_Sid
temp=db.cursor()
temp.executescript('''
    DROP TABLE IF EXISTS Mid_Sid;
    CREATE TABLE Mid_Sid(mid INTEGER ,sid INTEGER,
                       latitude REAL, longitude REAL)''')


temp.executemany("""INSERT INTO Mid_Sid
                (mid ,sid, latitude , longitude) 
                VALUES ( ?, ?, ?, ? ) """,(lst_mid_sid))
                 

db.commit()

result=[] #final result

for row in temp.execute("""select sid stationID, latitude, longitude, count(*) NumofNearbyMonkepo
                from Mid_Sid
                group by sid
                order by NumofNearbyMonkepo DESC"""):
          
    result.append(row)

db.commit()    
temp.close()

for ele in result:            # show final result
    print ele

(1550, 35.91312, -79.048446, 4090)
(1529, 35.914951, -79.053346, 2652)
(1607, 35.912472, -79.054622, 2419)
(1560, 35.91075, -79.053626, 2383)
(1621, 35.91331, -79.055061, 2354)
(1631, 35.911293, -79.052368, 2193)
(1608, 35.911171, -79.051148, 2068)
(1557, 35.914915, -79.049916, 1997)
(1602, 35.912364, -79.05557, 1913)
(1628, 35.909378, -79.051145, 1805)
(1618, 35.912842, -79.054288, 1782)
(1598, 35.909696, -79.046723, 1777)
(1551, 35.913962, -79.050538, 1747)
(1559, 35.910723, -79.054127, 1735)
(1638, 35.927109, -79.026518, 1724)
(1633, 35.928041, -79.027461, 1620)
(1573, 35.911273, -79.04647, 1572)
(1566, 35.9105, -79.050046, 1495)
(1572, 35.913346, -79.051797, 1405)
(1514, 35.914371, -79.054019, 1404)
(1549, 35.914826, -79.052418, 1385)
(1540, 35.914434, -79.050973, 1373)
(1594, 35.911839, -79.049825, 1363)
(1563, 35.907843, -79.047398, 1356)
(1620, 35.912768, -79.045415, 1332)
(1619, 35.911192, -79.049326, 1323)
(1050, 35.929839, -79.033511, 1216)
(1611, 35.910402, -79.04857, 1204)


Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set.

You can submit it as many times as you would like, but only the last submitted version will be graded. If the last submission was after the due date the final score will be reduced accordingly. We do not need your database, since your Notebook should produce it.